In [ ]:
# main_script.py
import os
import pandas as pd
import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count
from data_processing import read_and_process

def main(folder_path):
    files = [os.path.join(folder_path, file) for file in os.listdir(folder_path)]
    with Pool(cpu_count()) as pool:
        results = pool.map(read_and_process, files)
    
    # 过滤掉处理失败的结果
    all_features = []
    all_targets = []
    for features, targets in results:
        if features is not None and targets is not None:
            all_features.extend(features)
            all_targets.extend(targets)

    return all_features, all_targets

# 读取数据
folder_path = '/Users/ekko/Downloads/coin_data'  # 更改为你的文件夹路径

# 主函数执行
all_features, all_targets = main(folder_path)

# 转换为DataFrame
features_df = pd.DataFrame(all_features)
targets_df = pd.DataFrame(all_targets)

# 合并特征和目标变量
data_df = pd.concat([features_df, targets_df], axis=1)

# 保存数据
data_df.to_csv('features_targets.csv', index=False)

print("Data collection complete.")


In [40]:
import polars as pl

# 读取 Excel 文件中的所有工作表
file_path = 'features_targets.csv'
df = pl.read_csv(file_path)  # sheet_id=None 表示读取所有工作表

print(df)

shape: (178_854, 55)
┌─────────┬───────────┬────────────┬────────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ open_ti ┆ open_toke ┆ current_ti ┆ current_to ┆ … ┆ max_increa ┆ max_decrea ┆ max_increa ┆ max_decrea │
│ me      ┆ n0        ┆ me         ┆ ken0       ┆   ┆ se_3600s   ┆ se_3600s   ┆ se_all     ┆ se_all     │
│ ---     ┆ ---       ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│ str     ┆ i64       ┆ str        ┆ i64        ┆   ┆ f64        ┆ f64        ┆ f64        ┆ f64        │
╞═════════╪═══════════╪════════════╪════════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 2024-06 ┆ 100000000 ┆ 2024-06-09 ┆ 1000000000 ┆ … ┆ 6.0163e11  ┆ 1.0000e10  ┆ 6.0163e11  ┆ 1.0000e10  │
│ -09 03: ┆ 00        ┆ 03:26:32   ┆ 0          ┆   ┆            ┆            ┆            ┆            │
│ 26:32   ┆           ┆            ┆            ┆   ┆            ┆            ┆            ┆            │
│ 2024-06 ┆ 100000000 ┆ 2

In [46]:
data = df.__copy__()

In [47]:
# 特征列表
features_to_transform = [
    'current_token0',
    'token0_value_5s',
    'token0_value_10s',
    'token0_value_30s',
    'token0_value_60s',
    'token0_value_120s',
    'token0_value_300s',
    'token0_value_600s',
    'token0_value_1200s',
    'token0_value_1800s',
    'token0_value_3600s',
    'inflow_5s',
    'outflow_5s',
    'inflow_10s',
    'outflow_10s',
    'inflow_30s',
    'outflow_30s',
    'inflow_60s',
    'outflow_60s',
    'inflow_120s',
    'outflow_120s',
    'inflow_300s',
    'outflow_300s',
    'inflow_600s',
    'outflow_600s',
    'inflow_1200s',
    'outflow_1200s',
    'inflow_1800s',
    'outflow_1800s',
    'inflow_3600s',
    'outflow_3600s',
    'max_increase_10s',
    'max_decrease_10s',
    'max_increase_30s',
    'max_decrease_30s',
    'max_increase_60s',
    'max_decrease_60s',
    'max_increase_300s',
    'max_decrease_300s',
    'max_increase_600s',
    'max_decrease_600s',
    'max_increase_1800s',
    'max_decrease_1800s',
    'max_increase_3600s',
    'max_decrease_3600s',
    'max_increase_all',
    'max_decrease_all'
]

# 计算与初始值的比例
df = df.with_columns([
    (pl.col(col) / pl.col('open_token0')).alias(f'{col}_ratio') for col in features_to_transform
])

# 展示新生成的 DataFrame
print(df)

shape: (178_854, 102)
┌─────────┬───────────┬────────────┬────────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ open_ti ┆ open_toke ┆ current_ti ┆ current_to ┆ … ┆ max_increa ┆ max_decrea ┆ max_increa ┆ max_decrea │
│ me      ┆ n0        ┆ me         ┆ ken0       ┆   ┆ se_3600s_r ┆ se_3600s_r ┆ se_all_rat ┆ se_all_rat │
│ ---     ┆ ---       ┆ ---        ┆ ---        ┆   ┆ atio       ┆ atio       ┆ io         ┆ io         │
│ str     ┆ i64       ┆ str        ┆ i64        ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│         ┆           ┆            ┆            ┆   ┆ f64        ┆ f64        ┆ f64        ┆ f64        │
╞═════════╪═══════════╪════════════╪════════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 2024-06 ┆ 100000000 ┆ 2024-06-09 ┆ 1000000000 ┆ … ┆ 60.162618  ┆ 1.0        ┆ 60.162618  ┆ 1.0        │
│ -09 03: ┆ 00        ┆ 03:26:32   ┆ 0          ┆   ┆            ┆            ┆            ┆            │
│ 26:32   ┆           ┆ 

In [48]:
df.columns

['open_time',
 'open_token0',
 'current_time',
 'current_token0',
 'token0_value_5s',
 'token0_value_10s',
 'token0_value_30s',
 'token0_value_60s',
 'token0_value_120s',
 'token0_value_300s',
 'token0_value_600s',
 'token0_value_1200s',
 'token0_value_1800s',
 'token0_value_3600s',
 'inflow_5s',
 'outflow_5s',
 'inflow_10s',
 'outflow_10s',
 'inflow_30s',
 'outflow_30s',
 'inflow_60s',
 'outflow_60s',
 'inflow_120s',
 'outflow_120s',
 'inflow_300s',
 'outflow_300s',
 'inflow_600s',
 'outflow_600s',
 'inflow_1200s',
 'outflow_1200s',
 'inflow_1800s',
 'outflow_1800s',
 'inflow_3600s',
 'outflow_3600s',
 'num_addresses',
 'max_address_holding',
 'top_5_address_holding',
 'top_10_address_holding',
 'holding_entropy',
 'max_increase_10s',
 'max_decrease_10s',
 'max_increase_30s',
 'max_decrease_30s',
 'max_increase_60s',
 'max_decrease_60s',
 'max_increase_300s',
 'max_decrease_300s',
 'max_increase_600s',
 'max_decrease_600s',
 'max_increase_1800s',
 'max_decrease_1800s',
 'max_increase_

In [50]:
df['top_5_address_holding']

top_5_address_holding
f64
1.0
0.897436
0.998632
0.970929
0.919892
0.944594
0.919771
0.981616
0.99364


In [51]:
features = df[['num_addresses',
 'max_address_holding',
 'top_5_address_holding',
 'top_10_address_holding',
 'holding_entropy','current_token0_ratio',
 'token0_value_5s_ratio',
 'token0_value_10s_ratio',
 'token0_value_30s_ratio',
 'token0_value_60s_ratio',
 'token0_value_120s_ratio',
 'token0_value_300s_ratio',
 'token0_value_600s_ratio',
 'token0_value_1200s_ratio',
 'token0_value_1800s_ratio',
 'token0_value_3600s_ratio',
 'inflow_5s_ratio',
 'outflow_5s_ratio',
 'inflow_10s_ratio',
 'outflow_10s_ratio',
 'inflow_30s_ratio',
 'outflow_30s_ratio',
 'inflow_60s_ratio',
 'outflow_60s_ratio',
 'inflow_120s_ratio',
 'outflow_120s_ratio',
 'inflow_300s_ratio',
 'outflow_300s_ratio',
 'inflow_600s_ratio',
 'outflow_600s_ratio',
 'inflow_1200s_ratio',
 'outflow_1200s_ratio',
 'inflow_1800s_ratio',
 'outflow_1800s_ratio',
 'inflow_3600s_ratio',
 'outflow_3600s_ratio',
 'max_increase_10s_ratio',
 'max_decrease_10s_ratio',
 'max_increase_30s_ratio',
 'max_decrease_30s_ratio',
 'max_increase_60s_ratio',
 'max_decrease_60s_ratio',
 'max_increase_300s_ratio',
 'max_decrease_300s_ratio',
 'max_increase_600s_ratio',
 'max_decrease_600s_ratio',
 'max_increase_1800s_ratio',
 'max_decrease_1800s_ratio',
 'max_increase_3600s_ratio',
 'max_decrease_3600s_ratio',
 'max_increase_all_ratio',
 'max_decrease_all_ratio']]

In [52]:
features

num_addresses,max_address_holding,top_5_address_holding,top_10_address_holding,holding_entropy,current_token0_ratio,token0_value_5s_ratio,token0_value_10s_ratio,token0_value_30s_ratio,token0_value_60s_ratio,token0_value_120s_ratio,token0_value_300s_ratio,token0_value_600s_ratio,token0_value_1200s_ratio,token0_value_1800s_ratio,token0_value_3600s_ratio,inflow_5s_ratio,outflow_5s_ratio,inflow_10s_ratio,outflow_10s_ratio,inflow_30s_ratio,outflow_30s_ratio,inflow_60s_ratio,outflow_60s_ratio,inflow_120s_ratio,outflow_120s_ratio,inflow_300s_ratio,outflow_300s_ratio,inflow_600s_ratio,outflow_600s_ratio,inflow_1200s_ratio,outflow_1200s_ratio,inflow_1800s_ratio,outflow_1800s_ratio,inflow_3600s_ratio,outflow_3600s_ratio,max_increase_10s_ratio,max_decrease_10s_ratio,max_increase_30s_ratio,max_decrease_30s_ratio,max_increase_60s_ratio,max_decrease_60s_ratio,max_increase_300s_ratio,max_decrease_300s_ratio,max_increase_600s_ratio,max_decrease_600s_ratio,max_increase_1800s_ratio,max_decrease_1800s_ratio,max_increase_3600s_ratio,max_decrease_3600s_ratio,max_increase_all_ratio,max_decrease_all_ratio
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
4,0.434783,1.0,1.0,1.291317,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.00023,0.0,0.00023,0.0,0.00023,0.0,0.00023,0.0,0.00023,0.0,0.00023,0.0,0.00023,0.0,0.00023,0.0,0.00023,0.0,0.00023,0.0,1.00039,1.0,1.00039,1.0,1.182752,1.0,5.623137,1.0,18.410403,1.0,53.270829,1.0,60.162618,1.0,60.162618,1.0
7,0.25641,0.897436,1.0,1.779226,1.00023,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.00039,0.0,0.00039,0.0,0.00039,0.0,0.00039,0.0,0.00039,0.0,0.00039,0.0,0.00039,0.0,0.00039,0.0,0.00039,0.0,0.00039,0.0,1.00039,1.00035,1.00039,1.00035,1.182752,1.00035,5.842032,1.00035,18.410403,1.00035,53.270829,1.00035,60.162618,1.00035,60.162618,1.00035
11,0.285323,0.998632,1.283955,-inf,1.00039,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.00045,-0.0001,0.00045,-0.0001,0.00045,-0.0001,0.00045,-0.0001,0.00045,-0.0001,0.00045,-0.0001,0.00045,-0.0001,0.00045,-0.0001,0.00045,-0.0001,0.00045,-0.0001,1.00039,1.00035,1.00039,1.00035,1.182752,1.00035,6.795097,1.00035,18.510403,1.00035,53.270829,1.00035,60.162618,1.00035,60.162618,1.00035
12,0.277408,0.970929,1.248337,-inf,1.00035,1.00023,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.00023,-0.0001,0.00046,-0.0001,0.00046,-0.0001,0.00046,-0.0001,0.00046,-0.0001,0.00046,-0.0001,0.00046,-0.0001,0.00046,-0.0001,0.00046,-0.0001,0.00046,-0.0001,1.00038,1.00036,1.000381,1.00036,1.182752,1.00036,6.940097,1.00036,18.625674,1.00036,53.270829,1.00036,60.162618,1.00036,60.162618,1.00036
14,0.262826,0.919892,1.182718,-inf,1.00036,1.00023,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.00025,-0.0001,0.00048,-0.0001,0.00048,-0.0001,0.00048,-0.0001,0.00048,-0.0001,0.00048,-0.0001,0.00048,-0.0001,0.00048,-0.0001,0.00048,-0.0001,0.00048,-0.0001,1.000381,1.00036,1.000381,1.00036,1.182752,1.00036,6.940097,1.00036,18.625674,1.00036,53.270829,1.00036,60.162618,1.00036,60.162618,1.00036
15,0.269884,0.944594,1.214478,-inf,1.00038,1.00035,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.00003,-0.00001,0.00048,-0.000109,0.00048,-0.000109,0.00048,-0.000109,0.00048,-0.000109,0.00048,-0.000109,0.00048,-0.000109,0.00048,-0.000109,0.00048,-0.000109,0.00048,-0.000109,1.000381,1.000371,1.000381,1.000371,1.182752,1.000371,6.940097,1.000371,18.625674,1.000371,53.270829,1.000371,60.162618,1.000371,60.162618,1.000371
16,0.262792,0.919771,1.182563,-inf,1.000371,1.00036,1.00023,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.00003,-0.00001,0.00026,-0.000109,0.00049,-0.000109,0.00049,-0.000109,0.00049,-0.000109,0.00049,-0.000109,0.00049,-0.000109,0.00049,-0.000109,0.00049,-0.000109,0.00049,-0.000109,1.000381,1.000371,1.004381,1.000371,1.182752,1.000371,6.949345,1.000371,18.625674,1.000371,53.270829,1.000371,60.162618,1.000371,60.162618,1.000371
17,0.913132,0.981616,1.013576,-inf,1.000381,1.000381,1.00038,1.0,1.0,

In [53]:
correlation_matrix = features.corr()

/Users/ekko/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2699: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


In [54]:
correlation_matrix

num_addresses,max_address_holding,top_5_address_holding,top_10_address_holding,holding_entropy,current_token0_ratio,token0_value_5s_ratio,token0_value_10s_ratio,token0_value_30s_ratio,token0_value_60s_ratio,token0_value_120s_ratio,token0_value_300s_ratio,token0_value_600s_ratio,token0_value_1200s_ratio,token0_value_1800s_ratio,token0_value_3600s_ratio,inflow_5s_ratio,outflow_5s_ratio,inflow_10s_ratio,outflow_10s_ratio,inflow_30s_ratio,outflow_30s_ratio,inflow_60s_ratio,outflow_60s_ratio,inflow_120s_ratio,outflow_120s_ratio,inflow_300s_ratio,outflow_300s_ratio,inflow_600s_ratio,outflow_600s_ratio,inflow_1200s_ratio,outflow_1200s_ratio,inflow_1800s_ratio,outflow_1800s_ratio,inflow_3600s_ratio,outflow_3600s_ratio,max_increase_10s_ratio,max_decrease_10s_ratio,max_increase_30s_ratio,max_decrease_30s_ratio,max_increase_60s_ratio,max_decrease_60s_ratio,max_increase_300s_ratio,max_decrease_300s_ratio,max_increase_600s_ratio,max_decrease_600s_ratio,max_increase_1800s_ratio,max_decrease_1800s_ratio,max_increase_3600s_ratio,max_decrease_3600s_ratio,max_increase_all_ratio,max_decrease_all_ratio
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.0,-0.000441,-0.000703,-0.000762,NaN,0.280608,0.280745,0.280946,0.281358,0.281956,0.283183,0.286926,0.292325,0.301939,0.307612,0.326299,0.02897,-0.048701,0.032016,-0.056298,0.079469,-0.088335,0.133117,-0.116678,0.176842,-0.154513,0.219962,-0.213307,0.248132,-0.262951,0.275075,-0.307456,0.28637,-0.323684,0.307,-0.349103,0.280565,0.28045,0.280506,0.280234,0.280392,0.279763,0.279517,0.276619,0.278464,0.272631,0.27513,0.254618,0.271517,0.227421,0.194793,-0.063504
-0.000441,1.0,0.997488,0.996103,NaN,-0.002965,-0.002964,-0.002964,-0.002963,-0.002963,-0.002962,-0.002959,-0.002954,-0.002938,-0.002917,-0.002843,-0.000525,0.000483,-0.000606,0.000541,-0.0012,0.000846,-0.001812,0.001103,-0.002301,0.001452,-0.002695,0.001924,-0.002861,0.002303,-0.002942,0.00256,-0.002942,0.002663,-0.002866,0.00266,-0.002965,-0.002965,-0.002966,-0.002964,-0.002967,-0.002963,-0.002969,-0.002955,-0.002972,-0.002945,-0.002977,-0.00291,-0.002981,-0.002854,-0.003005,-0.000715
-0.000703,0.997488,1.0,0.999847,NaN,-0.002864,-0.002863,-0.002863,-0.002862,-0.002862,-0.002861,-0.002857,-0.002852,-0.002836,-0.002815,-0.002742,-0.000507,0.000464,-0.000587,0.00052,-0.001161,0.000816,-0.001751,0.001063,-0.002223,0.001399,-0.002604,0.001855,-0.002764,0.00222,-0.002842,0.002469,-0.002845,0.00257,-0.00278,0.002578,-0.002864,-0.002863,-0.002865,-0.002863,-0.002865,-0.002862,-0.002868,-0.002854,-0.002871,-0.002845,-0.002876,-0.002811,-0.00288,-0.002757,-0.002903,-0.000692
-0.000762,0.996103,0.999847,1.0,NaN,-0.002834,-0.002833,-0.002833,-0.002832,-0.002832,-0.002831,-0.002827,-0.002821,-0.002805,-0.002784,-0.002711,-0.000502,0.000458,-0.000582,0.000513,-0.001149,0.000805,-0.001733,0.00105,-0.0022,0.001382,-0.002577,0.001834,-0.002736,0.002194,-0.002812,0.002438,-0.002815,0.00254,-0.002753,0.00255,-0.002834,-0.002833,-0.002835,-0.002833,-0.002835,-0.002832,-0.002838,-0.002824,-0.002841,-0.002815,-0.002846,-0.002782,-0.002849,-0.002729,-0.002873,-0.000687
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.280608,-0.002965,-0.002864,-0.002834,NaN,1.0,0.999925,0.99978,0.999516,0.999115,0.998323,0.995663,0.991761,0.984078,0.978198,0.961945,0.143165,-0.14509,0.165069,-0.155293,0.331368,-0.235679,0.504283,-0.302222,0.643491,-0.394088,0.768943,-0.52978,0.833865,-0.647284,0.884044,-0.746836,0.901448,-0.800585,0.917233,-0.851349,0.999975,0.99991,0.999972,0.999746,0.999969,0.999369,0.999919,0.996209,0.999832,0.992284,0.999275,0.972765,0.998296,0.937887,0.968023,0.153396
0.280745,-0.002964,-0.002863,-0.002833,NaN,0.999925,1.0,0.999854,0.999589,0.999